In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import numpy as np
import networkx as nx
import pylab as plt

eventsGermany = pd.read_json('/content/drive/My Drive/bachelor data/events/events_Germany.json')
matchesGermany = pd.read_json("/content/drive/My Drive/bachelor data/matches/matches_Germany.json")
tagsName = pd.read_csv("/content/drive/My Drive/bachelor data/tags2name.csv")
with open('/content/drive/My Drive/bachelor data/events/events_Germany.json') as f:
    d = json.load(f)
tags = json_normalize(data=d, record_path='tags', meta=['id'], record_prefix='_')
teams = pd.read_json("/content/drive/My Drive/bachelor data/teams.json")
eventsGermanyTags = pd.merge(eventsGermany,
                       tags,
                       on='id')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


In [ ]:
def checkZones2(dataframe, columnName):
  resultSeries = pd.Series([])
  i = -1
  for index, row in dataframe.iterrows():
    x = (row[columnName])['x']
    y = (row[columnName])['y']
    index = i + 1
    resultSeries = resultSeries.append(pd.Series([int(((x // 20) * 3) + 1 + (y // 33.3))]), ignore_index=True)
  return resultSeries

In [ ]:
# Filling 2D matrix of Barcelona team with corresponding passes
def get2dMatrix(passesDataframe, senderColumnName, RecipientColumnName):
  arr = np.arange(225).reshape(15,15)
  i = 0
  j = 0
  passCount = 0
  for i in range(15):
    for j in range(15):
      passCount = 0
      for index, row in passesDataframe.iterrows():
        if(row[senderColumnName] == i+1 and  row[RecipientColumnName] == j+1):
          passCount += 1
        arr[i][j] = passCount    
      j += 1
    i += 1 
  return arr

In [ ]:
def getTeamMatrix(teamId, teamMatchId):
  
  # Getting Successful passes of specific team in specific match
  successfulPasses = eventsGermanyTags[(eventsGermanyTags['teamId'] == teamId) & (eventsGermanyTags['matchId'] == teamMatchId) & (eventsGermanyTags['eventId'] == 8) & (eventsGermanyTags['_id'] == 1801)]
  successfulPasses.reset_index(level=0, inplace=True)

  # Adding Sender/Recipient positions
  successfulPasses2 = successfulPasses['positions'].sum()
  del successfulPasses2[1::2]
  x2 = pd.Series(successfulPasses2)
  successfulPassesSender = pd.concat([successfulPasses, pd.DataFrame(x2)], axis=1)
  successfulPassesSender.rename(columns = {0:'Sender'}, inplace = True)

  successfulPasses1 = successfulPassesSender['positions'].sum()
  del successfulPasses1[0::2]
  x = pd.Series(successfulPasses1)
  successfulPassesSenderRecipient = pd.concat([successfulPassesSender, pd.DataFrame(x)], axis=1)
  successfulPassesSenderRecipient.rename(columns = {0:'Recepient'}, inplace = True) 

  # Adding Sender/Recipient zones

  zoneSeriesSender = checkZones2(successfulPassesSenderRecipient, 'Sender')
  zoneDataframeSender = pd.DataFrame(zoneSeriesSender)
  zoneDataframeSender.rename(columns = {0:'SenderZone'}, inplace = True)

  zoneSeriesRecipient = checkZones2(successfulPassesSenderRecipient, 'Recepient')
  zoneDataframeRecipient = pd.DataFrame(zoneSeriesRecipient)
  zoneDataframeRecipient.rename(columns = {0:'RecipientZone'}, inplace = True)

  successfulPassesSenderRecipientZ = pd.concat([successfulPassesSenderRecipient, zoneDataframeSender], axis=1)
  successfulPassesSenderRecipientZone = pd.concat([successfulPassesSenderRecipientZ, zoneDataframeRecipient], axis=1)
  
  # returning the 2D passes Matrix
  return get2dMatrix(successfulPassesSenderRecipientZone, 'SenderZone', 'RecipientZone').flatten()

In [ ]:
def getTeamMatches(teamId):
  teamMatchesIds = np.array([])
  eventsGermanyTeam = eventsGermany[eventsGermany['teamId'] == teamId]
  teamMatchesIds = eventsGermanyTeam.matchId.unique()
  return teamMatchesIds

In [ ]:
def getTeamMatrices(teamId):
  teamMatches = getTeamMatches(teamId)
  teamMatrices = np.zeros((len(teamMatches), 225))
  for i in range(len(teamMatches)):
    teamMatrices[i] = getTeamMatrix(teamId, teamMatches[i])
  return teamMatrices

In [ ]:
bayernMunchenMatrices = np.zeros((38, 225))
bayernMunchenMatrices = getTeamMatrices(2444)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
bayernMunchenMatrices

array([[ 8.,  4.,  4., ...,  1.,  7., 12.],
       [ 2.,  2.,  2., ...,  4., 11., 11.],
       [ 4.,  2.,  3., ...,  0.,  5., 12.],
       ...,
       [ 6., 14.,  4., ...,  2.,  3.,  3.],
       [ 1.,  5.,  2., ...,  4.,  9., 25.],
       [ 5.,  6.,  3., ...,  1.,  8.,  9.]])

In [ ]:
schalkeMatrices = np.zeros((38, 225))
schalkeMatrices = getTeamMatrices(2449)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
schalkeMatrices

array([[ 4.,  2.,  2., ...,  0.,  0.,  6.],
       [ 4.,  3.,  1., ...,  2.,  6.,  8.],
       [ 8.,  5.,  1., ...,  2.,  3.,  5.],
       ...,
       [ 2.,  4.,  1., ...,  0.,  3.,  0.],
       [ 2.,  0.,  1., ...,  0.,  4.,  3.],
       [ 3.,  4.,  1., ...,  1.,  5., 10.]])

In [ ]:
hoffenheimMatrices = np.zeros((38, 225))
hoffenheimMatrices = getTeamMatrices(2482)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
hoffenheimMatrices

array([[ 0.,  2.,  3., ...,  1.,  3., 11.],
       [ 1.,  1.,  0., ...,  0.,  2.,  1.],
       [ 5.,  4.,  1., ...,  0.,  0.,  1.],
       ...,
       [ 5.,  5.,  0., ...,  2.,  3.,  3.],
       [ 4.,  5.,  0., ...,  3.,  6.,  5.],
       [ 6.,  5.,  2., ...,  1.,  0.,  1.]])

In [ ]:
dortmundMatrices = np.zeros((38, 225))
dortmundMatrices = getTeamMatrices(2447)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
dortmundMatrices

array([[ 2.,  2.,  0., ...,  5.,  5., 15.],
       [ 2.,  2.,  1., ...,  3.,  5.,  4.],
       [ 2.,  7.,  1., ...,  6.,  7.,  7.],
       ...,
       [ 1.,  6.,  4., ...,  1.,  3., 15.],
       [ 6.,  5.,  2., ...,  0.,  7.,  7.],
       [ 2.,  0.,  6., ...,  2.,  5.,  8.]])

In [ ]:
bayernLeverkusenMatrices = np.zeros((38, 225))
bayernLeverkusenMatrices = getTeamMatrices(2446)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
bayernLeverkusenMatrices

array([[ 8.,  3.,  2., ...,  0.,  9., 13.],
       [ 2.,  5.,  4., ...,  2.,  3.,  3.],
       [ 6.,  5.,  2., ...,  2.,  4.,  7.],
       ...,
       [ 2.,  5.,  1., ...,  4.,  6., 20.],
       [ 2.,  1.,  3., ...,  4.,  4., 13.],
       [ 2.,  5.,  0., ...,  7., 10., 32.]])

In [ ]:
leipzigMatrices = np.zeros((38, 225))
leipzigMatrices = getTeamMatrices(2975)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
leipzigMatrices

array([[ 6.,  0.,  3., ...,  0.,  8., 14.],
       [ 3.,  3.,  1., ...,  2.,  3.,  6.],
       [ 1.,  0.,  2., ...,  4.,  1., 13.],
       ...,
       [ 7.,  4.,  0., ...,  1.,  8.,  3.],
       [ 3.,  3.,  0., ...,  0.,  0.,  4.],
       [ 2.,  0.,  0., ...,  1.,  3., 13.]])

In [ ]:
stuttgurtMatrices = np.zeros((38, 225))
stuttgurtMatrices = getTeamMatrices(2445)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
stuttgurtMatrices

array([[ 0.,  0.,  2., ...,  0.,  2.,  5.],
       [ 2.,  1.,  0., ...,  0.,  4., 16.],
       [ 0.,  2.,  7., ...,  2.,  5., 15.],
       ...,
       [ 0.,  2.,  1., ...,  0.,  1.,  6.],
       [ 2.,  3.,  2., ...,  3.,  1.,  2.],
       [ 5.,  1.,  2., ...,  1.,  1.,  2.]])

In [ ]:
frankfurtMatrices = np.zeros((38, 225))
frankfurtMatrices = getTeamMatrices(2462)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
frankfurtMatrices

array([[ 1.,  6.,  3., ...,  4.,  1., 12.],
       [ 0.,  1.,  1., ...,  3.,  4.,  6.],
       [ 2.,  0.,  0., ...,  2.,  2.,  5.],
       ...,
       [ 1., 10.,  4., ...,  4.,  2.,  4.],
       [ 2.,  1.,  0., ...,  1.,  3.,  6.],
       [ 5.,  2.,  1., ...,  1.,  2.,  0.]])

In [ ]:
borussiaMGMatrices = np.zeros((38, 225))
borussiaMGMatrices = getTeamMatrices(2454)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
borussiaMGMatrices

array([[ 0.,  2.,  2., ...,  0.,  4.,  8.],
       [ 0.,  2.,  1., ...,  0.,  3.,  5.],
       [ 1.,  3.,  0., ...,  1.,  5., 28.],
       ...,
       [ 3., 13.,  1., ...,  2.,  5., 34.],
       [ 7.,  7.,  4., ...,  1.,  1.,  6.],
       [ 0.,  2.,  2., ...,  4.,  5.,  7.]])

In [ ]:
herthaMatrices = np.zeros((38, 225))
herthaMatrices = getTeamMatrices(2457)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
herthaMatrices

array([[ 8.,  4.,  3., ...,  0.,  5.,  3.],
       [ 2.,  1.,  0., ...,  1.,  2.,  6.],
       [ 2.,  0.,  2., ...,  0.,  5., 12.],
       ...,
       [ 4.,  6.,  1., ...,  3.,  3.,  8.],
       [ 1.,  3.,  2., ...,  5.,  5., 14.],
       [ 0.,  4.,  2., ...,  3.,  4.,  7.]])

In [ ]:
werderBermenMatrices = np.zeros((38, 225))
werderBermenMatrices = getTeamMatrices(2443)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
werderBermenMatrices

array([[ 7.,  1.,  3., ...,  3.,  6.,  9.],
       [ 1.,  2.,  1., ...,  1.,  3.,  4.],
       [ 0.,  0.,  1., ...,  1.,  1.,  8.],
       ...,
       [ 2.,  1.,  1., ...,  0.,  3.,  5.],
       [ 5.,  3.,  2., ...,  1.,  1.,  5.],
       [ 4.,  4.,  0., ...,  2.,  7., 14.]])

In [ ]:
augsburgMatrices = np.zeros((38, 225))
augsburgMatrices = getTeamMatrices(2481)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
augsburgMatrices

array([[8., 5., 5., ..., 4., 6., 9.],
       [3., 2., 2., ..., 4., 6., 7.],
       [0., 0., 1., ..., 1., 2., 5.],
       ...,
       [5., 6., 5., ..., 0., 0., 9.],
       [6., 8., 8., ..., 0., 4., 6.],
       [1., 3., 2., ..., 0., 4., 6.]])

In [ ]:
hannoverMatrices = np.zeros((38, 225))
hannoverMatrices = getTeamMatrices(2455)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
hannoverMatrices

array([[ 4.,  4.,  2., ...,  0.,  4.,  8.],
       [ 3.,  3.,  2., ...,  3.,  1.,  3.],
       [ 4.,  5.,  0., ...,  2.,  4., 16.],
       ...,
       [ 6.,  1.,  1., ...,  0.,  3.,  7.],
       [ 8.,  3.,  0., ...,  1.,  1.,  3.],
       [ 3.,  9.,  1., ...,  2.,  5.,  8.]])

In [ ]:
mainzMatrices = np.zeros((38, 225))
mainzMatrices = getTeamMatrices(2460)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
mainzMatrices

array([[ 3.,  2.,  3., ...,  0.,  1.,  1.],
       [ 5.,  5.,  1., ...,  2.,  1.,  2.],
       [ 4.,  5.,  3., ...,  0.,  3.,  1.],
       ...,
       [ 5.,  2.,  2., ...,  0.,  2.,  6.],
       [ 6.,  6.,  0., ...,  1.,  1.,  1.],
       [14.,  4.,  3., ...,  3.,  3.,  5.]])

In [ ]:
freiburgMatrices = np.zeros((38, 225))
freiburgMatrices = getTeamMatrices(2453)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
freiburgMatrices

array([[ 3.,  6.,  1., ...,  2.,  0.,  1.],
       [ 1.,  2.,  0., ...,  2.,  3.,  0.],
       [ 4.,  1.,  0., ...,  0.,  0.,  2.],
       ...,
       [ 2.,  0.,  1., ...,  2.,  1.,  2.],
       [ 5.,  1.,  4., ...,  3.,  5., 19.],
       [ 2.,  0.,  0., ...,  3.,  3.,  3.]])

In [ ]:
wolfsburgMatrices = np.zeros((38, 225))
wolfsburgMatrices = getTeamMatrices(2451)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
wolfsburgMatrices

array([[ 4.,  3.,  0., ...,  1.,  1.,  0.],
       [ 8.,  1.,  2., ...,  2.,  3.,  8.],
       [ 5.,  3.,  1., ...,  0.,  5., 11.],
       ...,
       [ 2.,  0.,  3., ...,  1.,  1.,  7.],
       [ 1.,  4.,  4., ...,  0.,  0.,  6.],
       [ 1.,  1.,  3., ...,  2.,  3., 13.]])

In [ ]:
hamburgerMatrices = np.zeros((38, 225))
hamburgerMatrices = getTeamMatrices(2450)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
hamburgerMatrices

array([[2., 2., 1., ..., 2., 3., 2.],
       [1., 2., 0., ..., 1., 0., 4.],
       [1., 2., 1., ..., 0., 3., 5.],
       ...,
       [6., 7., 7., ..., 0., 4., 6.],
       [6., 8., 3., ..., 0., 5., 9.],
       [6., 2., 3., ..., 1., 3., 4.]])

In [ ]:
kolnMatrices = np.zeros((38, 225))
kolnMatrices = getTeamMatrices(2463)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
kolnMatrices

array([[ 0.,  1.,  0., ...,  4.,  3.,  4.],
       [ 2.,  3.,  1., ...,  3.,  5.,  3.],
       [ 0.,  1.,  0., ...,  3.,  7.,  5.],
       ...,
       [10.,  5.,  6., ...,  2.,  4., 16.],
       [ 5.,  3.,  1., ...,  1.,  3.,  1.],
       [ 8.,  6.,  0., ...,  1.,  3.,  1.]])

In [ ]:
from google.colab import files
import pickle
with open('bundesliga.pickle', 'wb') as f:
  pickle.dump([bayernMunchenMatrices, schalkeMatrices, hoffenheimMatrices, dortmundMatrices, bayernLeverkusenMatrices, leipzigMatrices, stuttgurtMatrices, frankfurtMatrices, borussiaMGMatrices, herthaMatrices, werderBermenMatrices, augsburgMatrices, hannoverMatrices, mainzMatrices, freiburgMatrices, wolfsburgMatrices, hamburgerMatrices, kolnMatrices], f)
files.download('bundesliga.pickle')  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cols=[]
for i in range(612):
  if i<34:
    cols.append('red')
  else:
    if (i<68):  
      cols.append('black')
    else:
      if (i<102):
        cols.append('yellow')
      else:
        if (i<136):
          cols.append('blue')  
        else:
          if (i<170):
            cols.append('purple')
          else:
           if (i<204):
             cols.append('pink')  
           else:
            if (i<238):
              cols.append('navy')
            else:
              if (i<272):
               cols.append('turquoise')
              else:
                if (i<306):
                  cols.append('violet')
                else:
                  if (i<340):
                    cols.append('magenta')
                  else:
                    if (i<374):
                     cols.append('gray')
                    else:
                      if (i<408):
                       cols.append('cyan')
                      else:
                        if (i<442):
                          cols.append('brown')
                        else:
                          if (i<476):
                            cols.append('gold')
                          else:
                            if (i<510):
                              cols.append('orange')
                            else:
                              if (i<544):
                                cols.append('azure')  
                              else:
                                if (i<578):
                                  cols.append('salmon')
                                else:
                                  if (i<612):
                                    cols.append('darkviolet')  
                                  else:
                                    if (i<684):
                                      cols.append('lime')
                                    else:
                                      cols.append('silver')
                                     